# Testing with making the bubble plot

In [1]:
import pandas as pd
import matplotlib.image as mpimg
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import image, pyplot
# import logistic regression
from sklearn.linear_model import LogisticRegression


In [2]:
# read in data
df_raw = pd.read_csv("..\..\data\MC1\SensorDataProcessed.csv")
img = mpimg.imread('..\..\data\MC1\Lekagul Roadways.bmp')

In [3]:
# make a new dataframe with the columns: car-id, all gates as a binary one hot encoded column
df_encoded = df_raw.groupby(['car-id', 'gate-name']).size().unstack(fill_value=0)
df_encoded = df_encoded.reset_index()


In [6]:
df_encoded.iloc[1]

gate-name
car-id           20150001050042-811
camping0                          0
camping1                          0
camping2                          0
camping3                          0
camping4                          0
camping5                          0
camping6                          0
camping7                          2
camping8                          0
entrance0                         0
entrance1                         0
entrance2                         1
entrance3                         0
entrance4                         1
gate0                             0
gate1                             0
gate2                             0
gate3                             0
gate4                             0
gate5                             0
gate6                             0
gate7                             0
gate8                             0
general-gate0                     0
general-gate1                     0
general-gate2                     0
general-gate3     